In [13]:
# import packages
import ee
import geemap

import pandas as pd

In [2]:
# initialize the EE api
ee.Initialize()

In [3]:
# define EE datasets
agb = ee.ImageCollection("projects/sat-io/open-datasets/ESA/ESA_CCI_AGB")
y2y = ee.FeatureCollection("projects/ee-bermane/assets/y2y")
lc = ee.Image("USGS/NLCD_RELEASES/2020_REL/NALCMS")
rsr = ee.Image("projects/ee-bermane/assets/Root_shoot_ratio_Map_Merged")
spawn = ee.ImageCollection("NASA/ORNL/biomass_carbon_density/v1")
countries = ee.FeatureCollection("UN/Geodata/BNDA_simplified/current")
y2y_ecoregions = ee.FeatureCollection(
    "projects/ee-bermane/assets/y2y_ecoregions")
y2y_biomes = ee.FeatureCollection("projects/ee-bermane/assets/y2y_biomes")
soc_olm = ee.Image("projects/ee-bermane/assets/soc_0_1m_kg_m2_olm")
soc_soilgrids = ee.Image("projects/soilgrids-isric/ocs_mean")
soc_sothe = ee.Image("projects/ee-bermane/assets/McMaster_WWFCanada_soil_carbon1m_250m_kg-m2_version3")
soc_socs = ee.Image("projects/ee-bermane/assets/SOCS_0_1m_mg_ha_year_2010")

In [ ]:
# Define function to calculate zonal statistics
# specify image to use and reducer type (sum(), mean())
def calc_zonal_stats(feature, image_list, reducer_list):

    # loop through image and reducer list
    for image, reducer in zip(image_list, reducer_list):

        # calculate zonal stats across features
        stats = image.reduceRegion(
            reducer=reducer,
            geometry=feature.geometry(),
            crs=image.projection(),
            crsTransform=image.projection().getInfo().get('transform'),
            maxPixels=1e15
        )

        # add zonal stats to feature properties
        feature = feature.set(stats)
    return feature

# Define function to calculate feature area in km2
def calc_feature_area(feature):
    area_km2 = feature.geometry().area().divide(1e6)
    return feature.set({'area_km2_gee': area_km2})

# Define function to convert FeatureCollection to DataFrame
def fc_to_df(fc):
    """Extracts properties from FeatureCollection and converts to Pandas DataFrame."""
    features = fc.toList(fc.size()).getInfo()  # Convert to list
    data = [feature["properties"] for feature in features]  # Extract properties
    return pd.DataFrame(data)

In [5]:
# calc 2020 ESA CCI biomass
# grab the 2020 AGB images
agb_2020 = agb.filter(ee.Filter.stringContains("system:index", "2020")).first()

# add BGB to 2020 image using global rsr map
bio_2020 = agb_2020.addBands(agb_2020.select(['AGB']).multiply(rsr).rename('BGB')).select(['AGB', 'BGB'])

# add total carbon to 2020 image
bio_2020 = bio_2020.addBands(bio_2020.select(['AGB']).add(bio_2020.select(['BGB'])).rename('total_c'))

# rename bands
bio_2020 = bio_2020.rename(['agb_dens', 'bgb_dens', 'total_dens'])

# multiply values by 0.5 to get carbon density
bio_2020 = bio_2020.multiply(0.5).updateMask(lc.neq(18).And(lc.neq(19)))

# compute per-pixel area in ha
pixel_area_ha = ee.Image.pixelArea().divide(10000)

# calculate total biomass c per pixel and rename bands
bio_stock_2020 = bio_2020.multiply(pixel_area_ha).rename(['agb_c', 'bgb_c', 'total_c'])

In [6]:
# calc open land map soc
# multiply by 10 to get t/ha
# mask water/snow/ice
soc_olm = soc_olm.multiply(10).rename('soc_dens').updateMask(lc.neq(18).And(lc.neq(19)))

# multiply by pixel area to get total carbon per pixel
soc_olm_stock = soc_olm.multiply(pixel_area_ha).rename('total_olm_soc')

# reproject to match sothe to blend images
soc_olm_reproj_sothe = soc_olm.resample('bilinear').toFloat()

In [7]:
# calc sothe soc
# multiply by 10 to get t/ha
# mask water/snow/ice
soc_sothe = soc_sothe.multiply(10).rename('soc_dens').updateMask(lc.neq(18).And(lc.neq(19)))

# blend sothe and olm carbon across y2y
soc_blend = ee.ImageCollection([soc_olm_reproj_sothe, soc_sothe]).mosaic().rename('soc_blend_dens').reproject(
    crs=soc_sothe.projection(),
    crsTransform=soc_sothe.projection().getInfo().get('transform')
).updateMask(lc.neq(18).And(lc.neq(19)))

# multiply by pixel area to get total carbon per pixel
soc_blend_stock = soc_blend.multiply(pixel_area_ha).rename('total_blend_soc')

In [8]:
# calc socs soc
# mask water/snow/ice
soc_socs = soc_socs.rename('soc_socs_dens').updateMask(lc.neq(18).And(lc.neq(19)))

# multiply by pixel area to get total carbon per pixel
soc_socs_stock = soc_socs.multiply(pixel_area_ha).rename('total_soc_socs')

In [9]:
# check scale of layers
print('Biomass Carbon Density Scale: ', bio_2020.projection().nominalScale().getInfo())
print('Biomass Carbon Total Scale: ', bio_stock_2020.projection().nominalScale().getInfo())
print('SOC Density Sothe Scale: ', soc_sothe.projection().nominalScale().getInfo())
print('SOC Density Blend Scale: ', soc_blend.projection().nominalScale().getInfo())
print('SOC Total Blend Scale: ', soc_blend_stock.projection().nominalScale().getInfo())
print('SOC Density OLM Scale: ', soc_olm.projection().nominalScale().getInfo())

Biomass Carbon Density Scale:  98.95065848290943
Biomass Carbon Total Scale:  98.95065848290943
SOC Density Sothe Scale:  250.00021298953834
SOC Density Blend Scale:  250.00021298953834
SOC Total Blend Scale:  250.00021298953834
SOC Density OLM Scale:  231.91556871282307


In [ ]:
# define image and reducer list for zonal stats
img_list = [bio_stock_2020, bio_2020, soc_blend_stock, soc_blend, soc_socs_stock, soc_socs]
redu_list = [ee.Reducer.sum(), ee.Reducer.mean(), ee.Reducer.sum(), ee.Reducer.mean(), ee.Reducer.sum(), ee.Reducer.mean()]

# # Compute zonal statistics across Y2Y region
# y2y_carbon = y2y.map(lambda feature: calc_zonal_stats(feature, img_list, redu_list)).map(calc_feature_area)

# # Compute zonal statistics across ecoregions
# eco_carbon = y2y_ecoregions.map(lambda feature: calc_zonal_stats(feature, img_list, redu_list)).map(calc_feature_area)

# Compute zonal statistics across biomes
biome_carbon = fc_to_df(y2y_biomes.map(lambda feature: calc_zonal_stats(feature, img_list, redu_list)).map(calc_feature_area))

# Compute zonal statistics across Canada
canada = countries.filter(ee.Filter.eq('lbl_en', 'CANADA'))
img_list_can = [bio_2020, soc_blend, soc_socs]
redu_list_can = [ee.Reducer.mean(), ee.Reducer.mean(), ee.Reducer.mean()]
canada_carbon = canada.map(lambda feature: calc_zonal_stats(feature, img_list_can, redu_list_can)).map(calc_feature_area)

,BIOME_ID,BIOME_NAME,agb_c,agb_dens,area_km2,area_km2_gee,bgb_c,bgb_dens,soc_blend_dens,soc_socs_dens,total_blend_soc,total_c,total_dens,total_soc_socs
0,5,Temperate Conifer Forests,3.003759e+09,59.034848,604242.9,602219.807007,7.986885e+08,15.804549,198.731240,180.295012,1.153772e+10,3.792035e+09,75.194739,1.027897e+10
1,6,Boreal Forests/Taiga,1.553922e+09,45.408930,377124.8,375022.114150,3.841412e+08,11.588156,235.963042,229.658735,8.463470e+09,1.917579e+09,57.830834,8.398469e+09
2,8,"Temperate Grasslands, Savannas & Shrublands",2.812967e+07,11.316490,75578.6,75389.877363,7.343823e+06,3.231257,121.233915,125.577716,9.047571e+08,3.441819e+07,15.152255,9.420351e+08
3,11,Tundra,4.679915e+08,22.693174,283806.8,281997.069646,1.186128e+08,6.371054,269.511931,315.419028,7.467723e+09,5.393481e+08,28.839520,8.792563e+09
4,13,Deserts & Xeric Shrublands,3.623965e+06,4.886111,29129.2,29075.316155,9.345091e+05,1.543805,105.381060,112.973453,3.047621e+08,4.141109e+06,6.841070,3.269364e+08
